In [1]:
import sqlite3
import pandas as pd

testers=['S006','S008','S009','S010','S012','S013','S014','S015','S016','S017',\
         'S018','S019','S020','S021','S022','S023','S024','S025','S026','S027',\
            'S028','S029']

conn=sqlite3.connect('harth.db')

def dataOfTester(tester):
    return 'SELECT * FROM '+tester

def createDataframe(tester):
    tester_df=pd.read_sql_query(dataOfTester(tester),conn)
    if(tester=='S015' or tester=='S021'):
            tester_df=tester_df.drop(columns=['index'])
    if(tester=='S023'):
            tester_df=tester_df.drop(columns=['field1'])
    return tester_df

tester_df = [createDataframe(tester) for tester in testers]

tester_df[0]['timestamp']=pd.to_datetime(tester_df[0]['timestamp'])
timedelta_list = []

for df in tester_df:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    timedelta = df['timestamp'].iloc[1] - df['timestamp'].iloc[0]
    timedelta_list.append(timedelta)

print(timedelta_list)
for i in range(len(testers)):
    tester_df[i]['timestamp']=pd.to_datetime(tester_df[i]['timestamp'])
    interval=timedelta_list[i]
    tester_df[i]['sample']=(tester_df[i]['timestamp']-tester_df[i]['timestamp'].iloc[0])//interval
    tester_df[i].drop(columns=['timestamp'],inplace=True)



      
# tester_df[0]['sample']=(tester_df[0]['timestamp']-tester_df[0]['timestamp'].iloc[0])//interval
# tester_df[0].drop(columns=['timestamp'],inplace=True)



tester_df[4].head()



[Timedelta('0 days 00:00:00.010000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.019999995'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000'), Timedelta('0 days 00:00:00.020000')]


,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label,sample
0,-3.810360,0.165219,-0.663742,-0.919657,-0.759899,0.239633,6,0
1,-0.188988,-0.235452,-1.343093,-0.137463,-0.852652,-1.423640,6,1
2,-0.060000,0.008919,-0.876293,1.347957,-0.642281,-2.081678,6,2
3,-1.222393,0.028038,-0.460220,1.063477,-2.840740,-0.143171,6,3
4,-1.026113,-0.170239,-0.163068,-3.239610,0.924564,-0.720399,6,4


In [2]:
#lets create the rolling window
rolling_window=[0]*len(tester_df)
for i in range(len(tester_df)):
    rolling_window[i] = tester_df[i].iloc[:,:-2].rolling(window=10,center=True).mean()
    rolling_window[i]['label']=tester_df[i]['label']
    rolling_window[i].bfill(limit=5, inplace=True)
    rolling_window[i].ffill(limit=5, inplace=True)

for i in rolling_window:
    print(i.tail(10))


          back_x    back_y    back_z   thigh_x   thigh_y   thigh_z  label
408699 -0.718071  0.035292 -0.691135  0.063827  0.068942  1.002220      7
408700 -0.716116  0.034737 -0.691002  0.073453  0.069298  1.001427      7
408701 -0.717247  0.034221 -0.689879  0.061229  0.068797  1.003284      7
408702 -0.715693  0.034660 -0.691207  0.077474  0.069538  1.001981      7
408703 -0.717548  0.034044 -0.689069  0.054243  0.068358  1.003505      7
408704 -0.715584  0.035240 -0.693636  0.092638  0.070502  1.000754      7
408705 -0.715584  0.035240 -0.693636  0.092638  0.070502  1.000754      7
408706 -0.715584  0.035240 -0.693636  0.092638  0.070502  1.000754      7
408707 -0.715584  0.035240 -0.693636  0.092638  0.070502  1.000754      7
408708 -0.715584  0.035240 -0.693636  0.092638  0.070502  1.000754      7
          back_x    back_y    back_z   thigh_x   thigh_y   thigh_z  label
418979 -0.990182 -0.169308 -0.198136 -0.988800 -0.129428 -0.075978      3
418980 -0.986373 -0.163015 -0.199718 -

In [3]:
unified_df=pd.concat(rolling_window,ignore_index=True)
unified_df.head()

,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
0,-0.87638,0.114158,0.076672,-1.130216,-0.063653,-0.159632,6
1,-0.87638,0.114158,0.076672,-1.130216,-0.063653,-0.159632,6
2,-0.87638,0.114158,0.076672,-1.130216,-0.063653,-0.159632,6
3,-0.87638,0.114158,0.076672,-1.130216,-0.063653,-0.159632,6
4,-0.87638,0.114158,0.076672,-1.130216,-0.063653,-0.159632,6


In [4]:
reduced_df = unified_df.iloc[::10]
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646133 entries, 0 to 6461320
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   back_x   646133 non-null  float64
 1   back_y   646133 non-null  float64
 2   back_z   646133 non-null  float64
 3   thigh_x  646133 non-null  float64
 4   thigh_y  646133 non-null  float64
 5   thigh_z  646133 non-null  float64
 6   label    646133 non-null  int64  
dtypes: float64(6), int64(1)
memory usage: 34.5 MB


In [5]:
review=reduced_df['label']
reduced_df.drop(columns=['label'],inplace=True,errors='ignore')

C:\Users\gdste\AppData\Local\Temp\ipykernel_21132\3044613114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.drop(columns=['label'],inplace=True,errors='ignore')


In [6]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaled_df=reduced_df.copy()
columns_to_normalize = ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']
scaled_df[columns_to_normalize]=scaler.fit_transform(scaled_df[columns_to_normalize])
scaled_df.head()

,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z
0,0.028773,0.630492,0.712099,-1.088415,-0.338208,-0.874095
10,-0.408916,0.435352,0.748670,-0.771853,-0.095744,-0.965267
20,-0.358190,0.486492,0.749646,-0.739207,-0.172539,-1.058262
30,-0.353282,0.379491,0.727452,-0.748875,-0.048551,-1.062338
40,-0.356448,0.392260,0.734592,-0.744995,0.029453,-1.107746


In [13]:
from sklearn.cluster import DBSCAN
from sklearn.utils import resample

# Create DBSCAN object with increased eps value
dbscan = DBSCAN(eps=1.5, min_samples=150)
scaled_df.info()
smaller_df,smaller_review = resample(scaled_df,review, n_samples=50000, random_state=42)
smaller_df.info()
smaller_review.info()
# Fit the DBSCAN model on the scaled_df dataframe
dbscan.fit(smaller_df)

# Get the cluster labels
cluster_labels = dbscan.labels_
smaller_df['cluster']=cluster_labels
# Print the cluster labels
print(cluster_labels)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646133 entries, 0 to 6461320
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   back_x   646133 non-null  float64
 1   back_y   646133 non-null  float64
 2   back_z   646133 non-null  float64
 3   thigh_x  646133 non-null  float64
 4   thigh_y  646133 non-null  float64
 5   thigh_z  646133 non-null  float64
dtypes: float64(6)
memory usage: 29.6 MB
<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 1219580 to 4563660
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   back_x   50000 non-null  float64
 1   back_y   50000 non-null  float64
 2   back_z   50000 non-null  float64
 3   thigh_x  50000 non-null  float64
 4   thigh_y  50000 non-null  float64
 5   thigh_z  50000 non-null  float64
dtypes: float64(6)
memory usage: 2.7 MB
<class 'pandas.core.series.Series'>
Index: 50000 entries, 1219580 to 4563660
Seri

In [14]:
comparsion=pd.merge(smaller_df,smaller_review,left_index=True,right_index=True)
contingency_table=pd.crosstab(comparsion['label'],comparsion['cluster'])
print(contingency_table)

cluster   -1      0    1    2    3    4    5
label                                       
1         14  10034    0    0    0    0    0
2         90   2279    0    0    0    0    0
3          6   2065    0    0    0    0    0
4          2    647    0    0    0    0    0
5          4    519    0    0    0    0    0
6          0   6151    0    0    0    0    0
7          8  24122    0    0    1    0    0
8        228    777  940  586  719  172  210
13         1   3308    0    0    0    0    0
14        21    433    0    0    0    0    0
130        0    323    0    0    0    0    0
140        0     62    0    0    0    0    0


In [9]:
print(len(cluster_labels))
import numpy as np


unique_values = np.unique(cluster_labels)
num_unique_values = len(unique_values)

print("Number of unique values:", num_unique_values)

50000
Number of unique values: 5


In [10]:
for i in cluster_labels:
    print(i)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
